[View in Colaboratory](https://colab.research.google.com/github/somanshu-singla/Minor-Project/blob/master/Minor_Project_Duplicate_Question_Detector.ipynb)

In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
# Install other packages.
!pip3 install --quiet seaborn
# Install SentencePiece package
# SentencePiece package is needed for Universal Sentence Encoder Lite. We'll
# use it for all the text processing and sentence feature ID lookup.
!pip3 install --quiet sentencepiece

In [61]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#path for trainig, validation and testing data

train_data_file = '/content/drive/My Drive/project/train.csv'
validation_data_file = '/content/drive/My Drive/project/train.csv'
test_data_file = '/content/drive/My Drive/project/train.csv'

In [63]:
import pandas as pd

#using pandas to load data from csv file
train_data = pd.read_csv(train_data_file)
train_data = train_data.iloc[:, 0:6]
train_data.columns = ["id", "qid1", "qid2", "ques1", "ques2", "is_duplicate"]

validation_data = pd.read_csv(validation_data_file)
validation_data = train_data.iloc[:, 0:6]
validation_data.columns = ["id", "qid1", "qid2", "ques1", "ques2", "is_duplicate"]

#TODO: make change in test data processing
test_data = pd.read_csv(test_data_file)
test_data = train_data.iloc[:, :6]
test_data.columns = ["id", "qid1", "qid2", "ques1", "ques2", "is_duplicate"]

#checking the data
print(train_data["ques1"][0:10], train_data["ques2"][0:10], train_data["is_duplicate"][0:10])

0    What is the step by step guide to invest in sh...
1    What is the story of Kohinoor (Koh-i-Noor) Dia...
2    How can I increase the speed of my internet co...
3    Why am I mentally very lonely? How can I solve...
4    Which one dissolve in water quikly sugar, salt...
5    Astrology: I am a Capricorn Sun Cap moon and c...
6                                  Should I buy tiago?
7                       How can I be a good geologist?
8                      When do you use シ instead of し?
9    Motorola (company): Can I hack my Charter Moto...
Name: ques1, dtype: object 0    What is the step by step guide to invest in sh...
1    What would happen if the Indian government sto...
2    How can Internet speed be increased by hacking...
3    Find the remainder when [math]23^{24}[/math] i...
4              Which fish would survive in salt water?
5    I'm a triple Capricorn (Sun, Moon and ascendan...
6    What keeps childern active and far from phone ...
7            What should I do to be a 

In [64]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk.tokenize import word_tokenize
import string

def text_preprocess(text):
    text = [word.lower() for word in word_tokenize(text) if word not in string.punctuation]
    return text

In [66]:
#testing text_preprocess
print(text_preprocess("This is Somanshu's notebook!"))

['this', 'is', 'somanshu', "'s", 'notebook']


In [67]:
train_QUES1_LIST = list(map(lambda text: text_preprocess(text), train_data["ques1"]))
train_QUES2_LIST = list(map(lambda text: text_preprocess(text), train_data["ques2"]))

validation_QUES1_LIST = list(map(lambda text: text_preprocess(text), validation_data["ques1"]))
validation_QUES2_LIST = list(map(lambda text: text_preprocess(text), validation_data["ques2"]))

test_QUES1_LIST = list(map(lambda text: text_preprocess(text), test_data["ques1"]))
test_QUES2_LIST = list(map(lambda text: text_preprocess(text), test_data["ques2"]))

#checking processed list
print(train_QUES1_LIST[:3])
print(train_QUES2_LIST[:3])

[['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'], ['what', 'is', 'the', 'story', 'of', 'kohinoor', 'koh-i-noor', 'diamond'], ['how', 'can', 'i', 'increase', 'the', 'speed', 'of', 'my', 'internet', 'connection', 'while', 'using', 'a', 'vpn']]
[['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market'], ['what', 'would', 'happen', 'if', 'the', 'indian', 'government', 'stole', 'the', 'kohinoor', 'koh-i-noor', 'diamond', 'back'], ['how', 'can', 'internet', 'speed', 'be', 'increased', 'by', 'hacking', 'through', 'dns']]
